# Contract Analysis System - Testing Notebook

## Overview
This notebook provides comprehensive testing for the Contract Analysis System, which includes document reading, text processing, contract validation, and AI-powered analysis using local LLMs (Ollama).

## System Architecture
- **Document Reader**: Handles PDF, DOCX, and text files
- **Text Processor**: Advanced NLP for legal text analysis
- **Contract Validator**: Determines if document is a valid contract
- **LLM Manager**: Unified interface for Ollama, OpenAI, and Anthropic
- **Contract Classifier**: AI-powered contract categorization
- **Model Manager**: Handles model loading and caching


## Import Dependencies

In [1]:
import os
import sys
import json
from pathlib import Path
from pprint import pprint


# Add parent directory to path for module imports
sys.path.append('..')

# Import all system components
from utils.logger import log_info
from utils.logger import log_error
from config.risk_rules import ContractType
from utils.validators import ContractValidator
from utils.text_processor import TextProcessor
from utils.logger import ContractAnalyzerLogger
from services.term_analyzer import TermAnalyzer
from utils.document_reader import DocumentReader
from model_manager.llm_manager import LLMManager
from model_manager.llm_manager import LLMProvider
from model_manager.model_loader import ModelLoader
from services.clause_extractor import ClauseExtractor
from services.clause_extractor import ExtractedClause
from services.protection_checker import ProtectionChecker
from services.llm_interpreter import LLMClauseInterpreter
from services.negotiation_engine import NegotiationEngine
from services.risk_analyzer import MultiFactorRiskAnalyzer
from services.contract_classifier import ContractClassifier
from services.market_comparator import UniversalMarketComparator

print("✅ All modules imported successfully!")


✅ All modules imported successfully!


## Configuration

In [2]:
# Configuration settings
CONFIG = {"pdf_file_path"   : "../../../../Downloads/Satyaki Mitra - Employee Agreement - 2021.pdf",
          "use_spacy"       : True,  # Set to False if spaCy not installed
          "ollama_base_url" : "http://localhost:11434",
          "log_directory"   : "contract_analysis_logs",
         }

# Display configuration
print("Configuration Settings:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")
    

Configuration Settings:
  pdf_file_path: ../../../../Downloads/Satyaki Mitra - Employee Agreement - 2021.pdf
  use_spacy: True
  ollama_base_url: http://localhost:11434
  log_directory: contract_analysis_logs


## Document Reader Testing

In [3]:
# Initialize Document Reader
print("📄 STEP 1: Testing Document Reader\n")
print("=" * 60)

def test_document_reader(file_path):
    """
    Test the document reader with various file types
    """
    reader = DocumentReader()
    
    try:
        # Read the document
        file_contents = reader.read_file(file_path_or_bytes = file_path,
                                         file_type          = "pdf",
                                        )
        
        # Extract text content
        if isinstance(file_contents, dict):
            text     = file_contents.get('text', '') or file_contents.get('content', '')
            metadata = {k: v for k, v in file_contents.items() if k != 'text'}
        
        else:
            text     = str(file_contents)
            metadata = dict()
            
        
        # Display results
        print(f"✅ Document read successfully!\n")
        print(f"📊 Text length: {len(text):,} characters\n")
        print(f"\nText preview:\n")
        print("-" * 50)
        print(text)
        print("-" * 50)
        print("\n\n")
        
        if metadata:
            print(f"📋 Metadata: {list(metadata.keys())}")
        
        return text, metadata
        
    except Exception as e:
        print(f"❌ Error reading document: {e}")
        return None, None


# Test with configured PDF file
document_text, document_metadata = test_document_reader(file_path = CONFIG["pdf_file_path"])

if not document_text:
    print("⚠️  No text extracted.")


📄 STEP 1: Testing Document Reader

✅ Document read successfully!

📊 Text length: 26,469 characters


Text preview:

--------------------------------------------------
Itobuz Technologies Pvt. Ltd. Private and Confidential

Agreement of Employment

This Agreement for service (hereinafter referred to as “Agreement”) made and entered into on the 01st day of December 2022, by

and between Itobuz Technologies Private Limited a Company registered under the Companies Act 2013 having registered office at

STEP, IIT KHARAGPUR, P.S.- IIT KHARAGPUR, KHARAGPUR, WEST BENGAL 721302, INDIA, CIN No. U72200WB2010PTC150305

(hereinafter referred to as the “Employer”)

And

Satyaki Mitra son of Debdas Mitra residing at 28/6, Nabin Senapati Lane, P.O. - Baishnab Para Bazaar, P.S. - Shibpur, Howrah,

West Bengal - 711101 (hereinafter referred to as the “Employee”)

RECITALS

A. The Employer is engaged in the business of Software development and Information Technology based services (hereinafter

referred t

## Contract Validation Testing

In [4]:
# Initialize Contract Validator
print("\n🔍 STEP 2: Testing Contract Validator\n")
print("=" * 60)

def test_contract_validation(text):
    """
    Test if the document is a valid contract
    """
    validator = ContractValidator()
    
    print("📋 Running contract validation...")
    
    # Test 1: File integrity check (simulated)
    file_valid, file_message                         = True, "File check simulated - always passes in notebook"
    print(f"📁 File Integrity: {file_valid} - {file_message}\n")
    
    # Test 2: Contract validation
    is_contract, validation_type, validation_message = validator.is_valid_contract(text = text)
    
    print(f"📑 Contract Validation Results:")
    print(f"   Is Contract: {is_contract}")
    print(f"   Confidence: {validation_type}")
    print(f"   Message: {validation_message}\n")
    
    # Test 3: Detailed validation report
    validation_report                                = validator.get_validation_report(text = text)
    
    print(f"\n📊 Detailed Validation Report:")
    print(f"   Total Score: {validation_report['scores']['total']}")
    print(f"   Found Indicators: {len(validation_report['found_indicators'])}")
    print(f"   Anti-patterns: {len(validation_report['found_anti_patterns'])}\n")
    
    # Display key features
    features                                         = validation_report['features']
    print(f"   Key Features:")
    for feature, value in features.items():
        print(f"     - {feature}: {value}")
    
    # Display top indicators
    if validation_report['found_indicators']:
        print(f"   Top Indicators: {validation_report['found_indicators'][:5]}")
    
    return validation_report

# Run validation test
validation_report = test_contract_validation(text = document_text)



🔍 STEP 2: Testing Contract Validator

📋 Running contract validation...
📁 File Integrity: True - File check simulated - always passes in notebook

📑 Contract Validation Results:
   Is Contract: True
   Confidence: high_confidence
   Message: Strong contract indicators detected (score: 95). Found: agreement, contract, party, parties, hereinafter. This is highly likely a legal contract.


📊 Detailed Validation Report:
   Total Score: 88
   Found Indicators: 10
   Anti-patterns: 1

   Key Features:
     - has_signature_block: False
     - has_effective_date: True
     - has_party_identification: True
   Top Indicators: ['agreement', 'contract', 'party', 'parties', 'hereinafter']


## Text Processing Testing

In [5]:
# Initialize Text Processor
print("\n📝 STEP 3: Testing Text Processor")
print("=" * 60)

def test_text_processing(text: str, use_spacy: bool = True):
    """
    Test advanced text processing capabilities
    """
    processor = TextProcessor(use_spacy = use_spacy)
    
    print("🔧 Initializing text processor...")
    
    # Test 1: Basic text statistics
    print("\n📊 1. Text Statistics:")
    text_statistics = processor.get_text_statistics(text = text)
    
    for key, value in text_statistics.items():
        print(f"   {key.replace('_', ' ').title()}: {value}")
    
    # Test 2: Legal entity extraction
    print("\n🏛️  2. Legal Entity Extraction:")
    legal_entities      = processor.extract_legal_entities(text = text)
    legal_entity_counts = {k: len(v) for k, v in legal_entities.items() if v}
    
    for entity_type, count in legal_entity_counts.items():
        print(f"   {entity_type.title()}: {count} found\n")
        
        if ((entity_type in ['parties', 'dates', 'amounts']) and legal_entities[entity_type]):
            # Show first 10 samples
            samples = legal_entities[entity_type][:10]  
            print(f"     Samples: {samples}")
    
    # Test 3: Sentence extraction
    print("\n📝 3. Sentence Analysis:")
    sentences = processor.extract_sentences_advanced(text = text)
    print(f"   Total sentences: {len(sentences)}")
    
    if sentences:
        print("   First 10 sentences with entities:")
        for i, sent in enumerate(sentences[:10]):
            print(f"     {i+1}. {sent['text']}\n")
            if sent['entities']:
                print(f"        Entities: {sent['entities']}")
    
    # Test 4: Text chunking for analysis
    print("\n📦 4. Text Chunking:")
    chunks = processor.chunk_text_for_embedding(text       = text, 
                                                chunk_size = 512, 
                                                overlap    = 50,
                                               )
    
    print(f"   Created {len(chunks)} chunks for analysis\n")
    
    if chunks:
        print(f"   First chunk preview:")
        print(f"     Text: {chunks[0]['text']}\n")
        print(f"     Word count: {chunks[0]['word_count']}\n")
        print(f"     Sentences: {chunks[0]['start_sentence']}-{chunks[0]['end_sentence']}\n\n")
    
    # Test 5: Specialized legal extraction
    print("\n💰 5. Financial & Legal Elements:")
    monetary_amounts = processor.extract_monetary_amounts(text = text)
    durations        = processor.extract_durations(text = text)
    percentages      = processor.extract_percentages(text = text)
    
    print(f"   Monetary amounts: {monetary_amounts}")
    print(f"   Durations: {durations}")
    print(f"   Percentages: {percentages}\n\n")
    
    return {'statistics'     : text_statistics,
            'legal_entities' : legal_entities,
            'sentences'      : sentences,
            'chunks'         : chunks,
           }

# Run text processing test
processing_results = test_text_processing(text      = document_text, 
                                          use_spacy = CONFIG["use_spacy"],
                                         )



📝 STEP 3: Testing Text Processor
[TextProcessor] spaCy model loaded successfully
🔧 Initializing text processor...

📊 1. Text Statistics:
   Character Count: 26469
   Word Count: 4061
   Sentence Count: 168
   Paragraph Count: 264
   Avg Words Per Sentence: 24.172619047619047
   Avg Chars Per Word: 6.517852745629155
   Language: en

🏛️  2. Legal Entity Extraction:
   Parties: 5 found

     Samples: ['Employer', 'Employee', 'Client', 'Itobuz Technologies Private Limited', 'The Company']
   Dates: 1 found

     Samples: ['01-12-2022']
   References: 15 found


📝 3. Sentence Analysis:
   Total sentences: 218
   First 10 sentences with entities:
     1. Itobuz Technologies Pvt.

        Entities: [('Itobuz Technologies Pvt', 'ORG')]
     2. Ltd. Private and Confidential

Agreement of Employment

This Agreement for service (hereinafter referred to as “Agreement”) made and entered into on the 01st day of December 2022, by

and between Itobuz Technologies Private Limited a Company registered 

## LLM Manager Testing (Ollama)

In [6]:
# Initialize LLM Manager
print("\n💬 STEP 4: Testing LLM Manager with Ollama")
print("=" * 60)

def test_llm_manager(text_snippet: str):
    """
    Test LLM capabilities using Ollama
    """
    print("🚀 Initializing LLM Manager...")
    
    try:
        # Initialize LLM manager
        llm_manager         = LLMManager(default_provider = LLMProvider.OLLAMA,
                                         ollama_base_url  = CONFIG["ollama_base_url"],
                                        )
        
        # Check available providers
        available_providers = llm_manager.get_available_providers()
        
        print(f"✅ Available LLM Providers: {[p.value for p in available_providers]}")
        
        if LLMProvider.OLLAMA not in available_providers:
            print("❌ Ollama not available. Please ensure Ollama is running.")
            print("   Start Ollama: ollama serve")
            return None
        
        # Check available models
        ollama_models = llm_manager.list_ollama_models()
        print(f"📚 Available Ollama Models: {ollama_models}")
        
        if not ollama_models:
            print("⚠️  No Ollama models found. Pull a model: ollama pull llama2")
            return None
        
        # Test 1: Basic completion
        print("\n🧪 1. Testing Basic Completion:")
        test_prompt = "What are the key elements of an employment agreement?"
        
        response    = llm_manager.complete(prompt      = test_prompt,
                                           provider    = LLMProvider.OLLAMA,
                                           temperature = 0.1,
                                           max_tokens  = 512,
                                          )
        
        print(f"   Prompt: {test_prompt}")
        print(f"   Response: {response.text}")
        print(f"   Success: {response.success}")
        print(f"   Tokens: {response.tokens_used}")
        print(f"   Latency: {response.latency_seconds:.2f}s")
        
        # Test 2: Contract analysis
        print("\n📑 2. Testing Contract Analysis:")
        analysis_prompt  = f"""
                                Analyze this contract text and identify:
                                1. The parties involved
                                2. Main obligations  
                                3. Key financial terms
                                4. Duration/term
                                
                                Contract text: {text_snippet[:2000]}  # Limit to first 2000 chars
                            """
        
        analysis_response = llm_manager.complete(prompt      = analysis_prompt,
                                                 provider    = LLMProvider.OLLAMA,
                                                 temperature = 0.1,
                                                 max_tokens  = 500,
                                                )
        
        print("\n   Contract Analysis Results:")
        print("   " + "=" * 40)
        print(analysis_response.text)
        print("   " + "=" * 40)
        
        # Test 3: FIXED JSON structured output
        print("\n📋 3. Testing Structured Output:")
        try:
            # Create a better prompt with the actual contract text
            json_prompt      = f"""
                                    Extract the key parties and their roles from the following contract text. Return ONLY valid JSON with no additional text.
                                
                                    Contract Text:
                                    {text_snippet[:1500]}
                                
                                    Return JSON format:
                                    {{
                                      "parties": [
                                        {{
                                          "name": "party_name",
                                          "role": "party_role",
                                          "type": "individual/organization"
                                        }}
                                      ],
                                       "agreement_type": "type_of_agreement",
                                       "effective_date": "date_if_mentioned"
                                    }}
                                """
            
            # Use a more detailed schema description
            schema_description = """
                                    JSON schema with:
                                    - parties: array of objects with name, role, and type
                                    - agreement_type: string describing the type of agreement
                                    - effective_date: string with the effective date if mentioned
                                    - compensation: object with salary/amount details if mentioned
                                 """
            
            json_response      = llm_manager.generate_structured_json(prompt             = json_prompt,
                                                                      schema_description = schema_description,
                                                                      provider           = LLMProvider.OLLAMA,
                                                                      max_tokens         = 1024,
                                                                      temperature        = 0.1,
                                                                     )
            
            print("   Structured JSON Output:")
            pprint(json_response)
            
            # Validate the response
            if ((json_response.get('parties') == ['Alice', 'Bob']) or (json_response.get('roles') == ['Seller', 'Buyer'])):
                print("\n   ⚠️  WARNING: Model generated generic placeholder data!")
                print("   This indicates the model didn't properly analyze the contract.")
                
        except Exception as e:
            print(f"   JSON generation failed: {e}")
            
            # Fallback: Try manual JSON parsing with a simpler approach
            print("\n   🔧 Trying alternative JSON extraction...")
            try:
                fallback_prompt  = f"""
                                        Based on this contract text, extract the parties and their roles in JSON format:
                                        
                                        {text_snippet[:1000]}
                                        
                                        Return ONLY JSON, no other text. Example format:
                                        {{
                                          "parties": [
                                            {{
                                              "name": "Company Name", 
                                              "role": "Employer"
                                            }},
                                            {{
                                              "name": "Employee Name", 
                                              "role": "Employee" 
                                            }}
                                          ]
                                        }}
                                    """
                
                fallback_response = llm_manager.complete(prompt      = fallback_prompt,
                                                         provider    = LLMProvider.OLLAMA,
                                                         temperature = 0.1,
                                                         max_tokens  = 500,
                                                         json_mode   = True,
                                                        )
                
                if fallback_response.success:
                    # Try to parse the response as JSON
                    try:
                        # Clean the response
                        json_text   = fallback_response.text.strip()
                        json_text   = json_text.replace('```json', '').replace('```', '').strip()
                        
                        parsed_json = json.loads(json_text)
                        print("   Alternative JSON Output:")
                        pprint(parsed_json)
                        
                    except json.JSONDecodeError:
                        print("   Could not parse JSON from response:")
                        print(f"   Response: {fallback_response.text}")
                        
            except Exception as fallback_error:
                print(f"   Alternative approach also failed: {fallback_error}")
        
        return llm_manager
        
    except Exception as e:
        print(f"❌ LLM Manager test failed: {e}")
        import traceback
        print(f"Detailed error: {traceback.format_exc()}")
        return None


# Run LLM test with the extracted document text
llm_manager = test_llm_manager(text_snippet = document_text)



💬 STEP 4: Testing LLM Manager with Ollama
🚀 Initializing LLM Manager...
[Logger] Logging initialized. Logs: logs
✅ Available LLM Providers: ['ollama']
📚 Available Ollama Models: ['llama3:8b', 'mistral:7b', 'deepseek-r1:32b', 'qwen3:32b']

🧪 1. Testing Basic Completion:
   Prompt: What are the key elements of an employment agreement?
   Response:  An employment agreement, also known as an offer letter or contract of employment, is a legally binding document that outlines the terms and conditions of employment between an employer and an employee. Here are some key elements typically included in an employment agreement:

1. Parties involved: The names of the employer and employee, along with their respective addresses and contact information.

2. Position and duties: A clear description of the job title, position, and the main responsibilities and duties associated with the role.

3. Term of employment: The start date, duration of the employment (e.g., fixed term or indefinite), and any 

## Contract Classification Testing

In [7]:
print("\n🏷️ STEP 5: Testing Contract Classifier")
print("=" * 60)

def test_contract_classification(text):
    """
    Test AI-powered contract classification
    """
    print("🎯 Initializing Contract Classifier...")
    
    try:
        # Initialize model loader and classifier
        model_loader = ModelLoader()
        classifier   = ContractClassifier(model_loader)
        
        print("✅ Models loaded successfully!")
        
        # Test 1: Single category classification
        print("\n🔍 1. Single Category Classification:")
        classification = classifier.classify_contract(contract_text = text)
        
        print(f"   Primary Category: {classification.category}")
        print(f"   Subcategory: {classification.subcategory}")
        print(f"   Confidence: {classification.confidence:.2f}")
        
        print(f"   Reasoning:")
        for reason in classification.reasoning:
            print(f"     - {reason}")
        
        print(f"   Detected Keywords: {classification.detected_keywords}")
        
        # Test 2: Multi-label classification
        print("\n🏷️  2. Multi-Label Classification:")
        multi_categories = classifier.classify_multi_label(text      = text, 
                                                           threshold = 0.5,
                                                          )
        
        print(f"   Found {len(multi_categories)} relevant categories:")
        for i, category in enumerate(multi_categories):
            print(f"     {i+1}. {category.category} (confidence: {category.confidence:.2f})")
            if category.subcategory:
                print(f"        Subcategory: {category.subcategory}")
        
        # Test 3: Category descriptions
        print("\n📚 3. Available Categories:")
        all_categories = classifier.get_all_categories()
        print(f"   Total categories: {len(all_categories)}")
        
        # Show descriptions for top categories
        for category in multi_categories[:3]:
            description = classifier.get_category_description(category = category.category)
            print(f"     - {category.category}: {description}")
        
        return {'primary_classification' : classification,
                'multi_categories'       : multi_categories,
                'all_categories'         : all_categories,
               }
        
    except Exception as e:
        print(f"❌ Contract classification failed: {e}")
        print("   This may be due to model download requirements.")
        import traceback
        traceback.print_exc()
        return None


# Run classification test
classification_results = test_contract_classification(text = document_text)



🏷️ STEP 5: Testing Contract Classifier
🎯 Initializing Contract Classifier...
✅ Models loaded successfully!

🔍 1. Single Category Classification:
   Primary Category: employment
   Subcategory: executive
   Confidence: 0.53
   Reasoning:
     - Strong keyword indicators for employment category (64% keyword match)
     - Specific subcategory identified: executive
     - Also contains elements of nda (secondary match: 0.52)
   Detected Keywords: ['employee', 'employment', 'job', 'position', 'salary', 'benefits', 'sick leave', 'probation', 'job description']

🏷️  2. Multi-Label Classification:
   Found 2 relevant categories:
     1. employment (confidence: 0.53)
        Subcategory: executive
     2. nda (confidence: 0.52)
        Subcategory: mutual_nda

📚 3. Available Categories:
   Total categories: 12
     - employment: Employment agreements governing employer-employee relationships
     - nda: Non-disclosure and confidentiality agreements


## Clause Extractor Testing

In [8]:
print("\n🔍 STEP 6: Testing Clause Extractor")
print("=" * 60)

def test_clause_extraction(text):
    """
    Test advanced clause extraction using Legal-BERT + structural patterns
    """
    print("🎯 Initializing Clause Extractor...")
    
    try:
        # Initialize model loader and clause extractor
        model_loader      = ModelLoader()
        
        # Get contract category from previous classification if available
        contract_category = None
        
        if classification_results and 'primary_classification' in classification_results:
            contract_category = classification_results['primary_classification'].category
        
        extractor = ClauseExtractor(model_loader      = model_loader, 
                                    contract_category = contract_category,
                                   )
        
        print("✅ Clause extractor initialized successfully!")
        
        # Test 1: Basic clause extraction
        print("\n📄 1. Basic Clause Extraction:")
        clauses = extractor.extract_clauses(contract_text = text, 
                                            max_clauses   = 50,
                                           )
        
        print(f"   Extracted {len(clauses)} clauses")
        
        # Show all clauses
        for i, clause in enumerate(clauses): 
            print(f"     {i+1}. [{clause.category}] {clause.reference}")
            print(f"        Confidence: {clause.confidence:.3f}")
            print(f"        Method: {clause.extraction_method}")
            print(f"        Text: {clause.text}")
            
            if clause.risk_indicators:
                print(f"        ⚠️  Risks: {clause.risk_indicators}")
            
            print()
        
        # Test 2: Extraction statistics
        print("\n📊 2. Extraction Statistics:")
        stats = extractor.get_extraction_stats(clauses)
        
        for key, value in stats.items():
            if isinstance(value, dict):
                print(f"   {key.replace('_', ' ').title()}:")
                for subkey, subvalue in value.items():
                    print(f"     - {subkey}: {subvalue}")
            
            else:
                print(f"   {key.replace('_', ' ').title()}: {value}")
        
        # Test 3: Category distribution
        print("\n🏷️  3. Category Distribution:")
        distribution = extractor.get_category_distribution(clauses)
        
        for category, count in distribution.items():
            print(f"   {category}: {count} clauses")
        
        # Test 4: High-risk clauses
        print("\n⚠️  4. High-Risk Clauses:")
        risky_clauses = extractor.get_high_risk_clauses(clauses)
        
        print(f"   Found {len(risky_clauses)} clauses with risk indicators")
        
        for i, clause in enumerate(risky_clauses):
            print(f"     {i+1}. {clause.reference} - {clause.category}")
            print(f"        Risks: {clause.risk_indicators}")
        
        return {'clauses'       : clauses,
                'stats'         : stats,
                'distribution'  : distribution,
                'risky_clauses' : risky_clauses,
               }
        
    except Exception as e:
        print(f"❌ Clause extraction failed: {e}")
        import traceback
        traceback.print_exc()
        return None


# Run clause extraction test
clause_results = test_clause_extraction(text = document_text)

# Store clauses for use in subsequent tests
if (clause_results and ('clauses' in clause_results)):
    extracted_clauses = clause_results['clauses']
    print(f"\n✅ Successfully extracted {len(extracted_clauses)} clauses for further analysis")

else:
    extracted_clauses = []
    print(f"\n⚠️  No clauses extracted - creating empty list for testing")



🔍 STEP 6: Testing Clause Extractor
🎯 Initializing Clause Extractor...
✅ Clause extractor initialized successfully!

📄 1. Basic Clause Extraction:
   Extracted 50 clauses
     1. [termination] 17.1
        Confidence: 0.767
        Method: structural
        Text: Notwithstanding anything contained herein the Employer reserves to terminate this Agreement at any point of time, for any reason whatsoever, with a notice of 15 days.
        ⚠️  Risks: ['terminate']

     2. [entire_agreement] 23.1
        Confidence: 0.752
        Method: structural
        Text: This written Agreement contains the sole and entire agreement between the parties and supersedes any other agreements between them.

     3. [termination] 10.3
        Confidence: 0.752
        Method: structural
        Text: The Employee agrees that in case of new business and or service opportunities which are similar in all material respects to the business of the Company in relation to in any geographical region other than tho

Traceback (most recent call last):
  File "/var/folders/jk/wxfv5xn16_b00bdt6v49v7640000gn/T/ipykernel_59783/2385891370.py", line 48, in test_clause_extraction
    stats = extractor.get_extraction_stats(clauses)
AttributeError: 'ClauseExtractor' object has no attribute 'get_extraction_stats'


## Risk Analyzer Testing

In [9]:
print("\n📊 STEP 7: Testing Risk Analyzer")
print("-" * 60)

def test_risk_analyzer(contract_text, clauses):
    """
    Test multi-factor risk analysis
    """
    print("🎯 Initializing Risk Analyzer...")
    
    try:
        # Initialize risk analyzer for employment contracts
        risk_analyzer = MultiFactorRiskAnalyzer(contract_type = ContractType.EMPLOYMENT)
        
        # Run comprehensive risk analysis
        print("🔍 Running multi-factor risk analysis...")
        risk_score    = risk_analyzer.analyze_risk(contract_text = contract_text,
                                                   clauses       = clauses,
                                                  )
        
        # Display results
        print(f"\n📈 RISK ANALYSIS RESULTS:")
        print(f"   Overall Score: {risk_score.overall_score}/100")
        print(f"   Risk Level: {risk_score.risk_level}")
        print(f"   High-Risk Categories: {len(risk_score.risk_factors)}")
        
        print(f"\n📋 Category Scores:")
        for category, score in risk_score.category_scores.items():
            level = "🔴" if (score >= 70) else "🟡" if (score >= 50) else "🟢"
            print(f"   {level} {category.replace('_', ' ').title()}: {score}/100")
        
        print(f"\n⚠️  Key Risk Factors:")
        for factor in risk_score.risk_factors:
            print(f"   - {factor.replace('_', ' ').title()}")
        
        if risk_score.benchmark_comparison:
            print(f"\n📊 Benchmark Comparison:")
            for item, comparison in risk_score.benchmark_comparison.items():
                print(f"   {item}: {comparison}")
        
        print(f"\n🔍 Detailed Breakdown:")
        for breakdown in risk_score.risk_breakdown[:3]:
            print(f"   📁 {breakdown.category}: {breakdown.score}/100")
            
            if breakdown.findings:
                print(f"      Finding: {breakdown.findings[0]}")
        
        return risk_score
        
    except Exception as e:
        print(f"❌ Risk analysis failed: {e}")
        import traceback
        traceback.print_exc()
        return None


# Run risk analyzer test with actual clauses
risk_results = test_risk_analyzer(contract_text = document_text,
                                  clauses       = extracted_clauses,
                                 )



📊 STEP 7: Testing Risk Analyzer
------------------------------------------------------------
🎯 Initializing Risk Analyzer...
🔍 Running multi-factor risk analysis...

📈 RISK ANALYSIS RESULTS:
   Overall Score: 12/100
   Risk Level: VERY LOW
   High-Risk Categories: 0

📋 Category Scores:
   🟢 Restrictive Covenants: 12/100
   🟢 Termination Rights: 12/100
   🟢 Penalties Liability: 12/100
   🟢 Compensation Benefits: 12/100
   🟢 Intellectual Property: 17/100

⚠️  Key Risk Factors:

🔍 Detailed Breakdown:
   📁 Intellectual Property: 17/100
   📁 Restrictive Covenants: 12/100
   📁 Termination Rights: 12/100


## Term Analyzer Testing

In [10]:
print("\n⚖️ STEP 8: Testing Term Analyzer")
print("-" * 60)

def test_term_analyzer(contract_text, clauses):
    """
    Test unfavorable terms detection
    """
    print("🎯 Initializing Term Analyzer...")
    
    try:
        # Initialize term analyzer
        term_analyzer     = TermAnalyzer()
        
        # Run unfavorable terms analysis
        print("🔍 Detecting unfavorable terms...")
        unfavorable_terms = term_analyzer.analyze_unfavorable_terms(contract_text = contract_text,
                                                                    clauses       = clauses,
                                                                   )
        
        # Display results
        print(f"\n📋 UNFAVORABLE TERMS ANALYSIS:")
        print(f"   Total Unfavorable Terms Found: {len(unfavorable_terms)}")
        
        # Severity distribution
        severity_dist = term_analyzer.get_severity_distribution(unfavorable_terms)
       
        print(f"\n📊 Severity Distribution:")
        for severity, count in severity_dist.items():
            icon = "🔴" if (severity == "critical") else "🟡" if (severity == "high") else "🟢"
            print(f"   {icon} {severity.title()}: {count} terms")
        
        # Category distribution
        category_dist = term_analyzer.get_category_distribution(unfavorable_terms)
        
        print(f"\n📁 Category Distribution:")
        for category, count in category_dist.items():
            print(f"   📂 {category}: {count} terms")
        
        # Show top unfavorable terms
        print(f"\n🚨 TOP UNFAVORABLE TERMS:")
        for i, term in enumerate(unfavorable_terms):
            print(f"\n   {i+1}. [{term.severity.upper()}] {term.term}")
            print(f"      Category: {term.category}")
            print(f"      Explanation: {term.explanation}")
            
            if term.suggested_fix:
                print(f"      💡 Suggested Fix: {term.suggested_fix}")
        
        return unfavorable_terms
        
    except Exception as e:
        print(f"❌ Term analysis failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run term analyzer test with actual clauses
term_results = test_term_analyzer(contract_text = document_text,
                                  clauses       = extracted_clauses,
                                 )



⚖️ STEP 8: Testing Term Analyzer
------------------------------------------------------------
🎯 Initializing Term Analyzer...
🔍 Detecting unfavorable terms...

📋 UNFAVORABLE TERMS ANALYSIS:
   Total Unfavorable Terms Found: 0

📊 Severity Distribution:
   🔴 Critical: 0 terms
   🟡 High: 0 terms
   🟢 Medium: 0 terms

📁 Category Distribution:

🚨 TOP UNFAVORABLE TERMS:


## Protection Checker Testing

In [11]:
print("\n🛡️ STEP 9: Testing Protection Checker")
print("-" * 60)

def test_protection_checker(contract_text, clauses):
    """
    Test missing protections detection
    """
    print("🎯 Initializing Protection Checker...")
    
    try:
        # Initialize protection checker
        protection_checker  = ProtectionChecker()
        
        # Run missing protections analysis
        print("🔍 Checking for missing protections...")
        missing_protections = protection_checker.check_missing_protections(contract_text = contract_text,
                                                                           clauses       = clauses,
                                                                          )
        
        # Display results
        print(f"\n🛡️ MISSING PROTECTIONS ANALYSIS:")
        print(f"   Total Missing Protections: {len(missing_protections)}")
        
        # Importance distribution
        importance_dist = protection_checker.get_importance_distribution(missing_protections)
        
        print(f"\n📊 Importance Distribution:")
        for importance, count in importance_dist.items():
            icon = "🔴" if (importance == "critical") else "🟡" if (importance == "high") else "🟢"
            print(f"   {icon} {importance.title()}: {count} protections")
        
        # Show critical missing protections
        critical_protections = protection_checker.get_critical_missing(missing_protections)
        
        print(f"\n🚨 CRITICAL MISSING PROTECTIONS:")
        for i, protection in enumerate(critical_protections[:3]):
            print(f"\n   {i+1}. {protection.protection}")
            print(f"      Category: {protection.category}")
            print(f"      Explanation: {protection.explanation}")
            print(f"      💡 Recommendation: {protection.recommendation}")
            
            if protection.examples:
                print(f"      📝 Example: {protection.examples[0]}")
        
        # Show all missing protections by category
        print(f"\n📁 ALL MISSING PROTECTIONS BY CATEGORY:")
        
        categories = set(p.category for p in missing_protections)
        
        for category in categories:
            category_protections = protection_checker.get_by_category(missing_protections, category)
            
            print(f"   📂 {category}: {len(category_protections)} missing")
            for prot in category_protections:
                print(f"      - {prot.protection} ({prot.importance})")
        
        return missing_protections
        
    except Exception as e:
        print(f"❌ Protection check failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run protection checker test with actual clauses
protection_results = test_protection_checker(contract_text = document_text,
                                             clauses       = extracted_clauses,
                                            )



🛡️ STEP 9: Testing Protection Checker
------------------------------------------------------------
🎯 Initializing Protection Checker...
🔍 Checking for missing protections...

🛡️ MISSING PROTECTIONS ANALYSIS:
   Total Missing Protections: 6

📊 Importance Distribution:
   🔴 Critical: 2 protections
   🟡 High: 3 protections
   🟢 Medium: 1 protections

🚨 CRITICAL MISSING PROTECTIONS:

   1. 'For Cause' Definition
      Category: termination
      Explanation: Without a clear definition of 'for cause', termination grounds are ambiguous and could be abused. This leaves you vulnerable to arbitrary termination claims.
      💡 Recommendation: Add: 'For Cause means: (a) gross negligence or willful misconduct, (b) material breach of this Agreement after 30-day written cure period, (c) conviction of a felony, or (d) fraud or embezzlement.'
      📝 Example: "For Cause" means (a) gross negligence, (b) willful misconduct, (c) material breach after cure period

   2. Liability Cap
      Category: liab

## LLM Interpreter Testing

In [12]:
print("\n💬 STEP 10: Testing LLM Interpreter")
print("-" * 60)

def test_llm_interpreter(clauses, llm_manager):
    """
    Test LLM-powered clause interpretation
    """
    if not llm_manager:
        print("⚠️  LLM Manager not available - skipping LLM Interpreter test")
        return None
        
    print("🎯 Initializing LLM Interpreter...")
    
    try:
        # Initialize LLM interpreter
        llm_interpreter = LLMClauseInterpreter(llm_manager = llm_manager)
        
        # Test with a few clauses
        test_clauses    = clauses if clauses else []
        
        if not test_clauses:
            print("⚠️  No clauses available for interpretation")
            return None
        
        print(f"🔍 Interpreting {len(test_clauses)} clauses with LLM...")
        interpretations = llm_interpreter.interpret_clauses(clauses     = test_clauses,
                                                            max_clauses = 50,
                                                           )
        
        # Display results
        print(f"\n💡 CLAUSE INTERPRETATIONS:")
        print(f"   Successfully Interpreted: {len(interpretations)} clauses")
        
        for i, interpretation in enumerate(interpretations):
            print(f"\n   {i+1}. [{interpretation.clause_reference}]")
            print(f"      📝 Summary: {interpretation.plain_english_summary}")
            print(f"      ⚖️  Favorability: {interpretation.favorability}")
            print(f"      🎯 Confidence: {interpretation.confidence:.2f}")
            
            if interpretation.key_points:
                print(f"      📋 Key Points:")
                for point in interpretation.key_points:
                    print(f"         • {point}")
            
            if interpretation.potential_risks:
                print(f"      ⚠️  Potential Risks:")
                for risk in interpretation.potential_risks:
                    print(f"         • {risk}")
        
        # Get unfavorable interpretations
        unfavorable = llm_interpreter.get_unfavorable_interpretations(interpretations)
        
        print(f"\n🚨 Unfavorable Interpretations: {len(unfavorable)}")
        
        # Get high-risk interpretations
        high_risk   = llm_interpreter.get_high_risk_interpretations(interpretations)
        
        print(f"⚠️  High-Risk Interpretations: {len(high_risk)}")
        
        return interpretations
        
    except Exception as e:
        print(f"❌ LLM interpretation failed: {e}")
        import traceback
        traceback.print_exc()
        return None


# Run LLM interpreter test with actual clauses
llm_interpretation_results = test_llm_interpreter(clauses     = extracted_clauses,
                                                  llm_manager = llm_manager,
                                                 )



💬 STEP 10: Testing LLM Interpreter
------------------------------------------------------------
🎯 Initializing LLM Interpreter...
⚠️  No clauses available for interpretation


## Negotiation Engine Testing

In [14]:
print("\n🤝 STEP 11: Testing Negotiation Engine")
print("-" * 60)

def test_negotiation_engine(risk_analysis, unfavorable_terms, missing_protections, clauses, llm_manager):
    """
    Test negotiation strategy generation
    """
    print("🎯 Initializing Negotiation Engine...")
    
    try:
        # Initialize negotiation engine
        negotiation_engine = NegotiationEngine(llm_manager = llm_manager)
        
        # Generate negotiation points
        print("💡 Generating negotiation strategy...")
        negotiation_points = negotiation_engine.generate_negotiation_points(risk_analysis       = risk_analysis,
                                                                            unfavorable_terms   = unfavorable_terms,
                                                                            missing_protections = missing_protections,
                                                                            clauses             = clauses,
                                                                            max_points          = 25,
                                                                           )
        
        # Display results
        print(f"\n🤝 NEGOTIATION STRATEGY:")
        print(f"   Total Negotiation Points: {len(negotiation_points)}")
        
        # Group by priority
        priority_groups = dict()
        
        for point in negotiation_points:
            if point.priority not in priority_groups:
                priority_groups[point.priority] = []
                
            priority_groups[point.priority].append(point)
        
        print(f"\n🎯 PRIORITIZED NEGOTIATION POINTS:")
        for priority in sorted(priority_groups.keys()):
            points = priority_groups[priority]
            priority_label = {1 : "🔴 CRITICAL", 
                              2 : "🟠 HIGH", 
                              3 : "🟡 MEDIUM", 
                              4 : "🟢 LOW",
                             }.get(priority, f"PRIORITY {priority}")
            
            print(f"\n   {priority_label} PRIORITY:")
            for i, point in enumerate(points):
                print(f"\n      {i+1}. {point.issue}")
                print(f"         📁 Category: {point.category}")
                print(f"         🎯 Difficulty: {point.estimated_difficulty}")
                print(f"         📝 Current: {point.current_language}")
                print(f"         💡 Proposed: {point.proposed_language}")
                print(f"         📚 Rationale: {point.rationale}")
                
                if point.fallback_position:
                    print(f"         🔄 Fallback: {point.fallback_position}")
        
        # Get critical points
        critical_points = negotiation_engine.get_critical_points(negotiation_points)
        
        print(f"\n🚨 CRITICAL NEGOTIATION POINTS: {len(critical_points)}")
        
        # Generate strategy document
        strategy_doc    = negotiation_engine.generate_negotiation_strategy_document(negotiation_points)
        
        print(f"\n📄 Strategy Document Length: {len(strategy_doc)} characters")
        
        # Show document preview
        print(f"\n📋 STRATEGY DOCUMENT PREVIEW:")
        doc_lines       = strategy_doc.split('\n')[:15]  # Show first 15 lines
        
        for line in doc_lines:
            # Only show non-empty lines
            if line.strip():  
                print(f"   {line}")
        
        return negotiation_points
        
    except Exception as e:
        print(f"❌ Negotiation engine failed: {e}")
        import traceback
        traceback.print_exc()
        return None


# Run negotiation engine test with actual data
negotiation_results = test_negotiation_engine(risk_analysis       = risk_results,
                                              unfavorable_terms   = term_results,
                                              missing_protections = protection_results,
                                              clauses             = extracted_clauses,
                                              llm_manager         = llm_manager,
                                             )



🤝 STEP 11: Testing Negotiation Engine
------------------------------------------------------------
🎯 Initializing Negotiation Engine...
💡 Generating negotiation strategy...

🤝 NEGOTIATION STRATEGY:
   Total Negotiation Points: 2

🎯 PRIORITIZED NEGOTIATION POINTS:

   🟠 HIGH PRIORITY:

      1. Add 'For Cause' Definition
         📁 Category: termination
         🎯 Difficulty: medium
         📝 Current: [NOT PRESENT IN CONTRACT]
         💡 Proposed: "For the purposes of this Agreement, 'For Cause' shall mean the occurrence of any of the following events: (a) Material breach of this Agreement by the Party, which breach is not cured within thirty (30) days after written notice from the non-breaching Party; (b) Theft or embezzlement by the Party; (c) Commission of a felony or any crime involving moral turpitude by the Party; (d) Bankruptcy or insolvency of the Party; (e) Conduct by the Party that materially and adversely affects the reputation
         📚 Rationale: Without a clear definiti

## Market Comparator Testing

In [15]:
print("\n🌍 STEP 12: Testing Universal Market Comparator")
print("-" * 60)

def test_universal_market_comparator(clauses, contract_type=ContractType.EMPLOYMENT):
    """
    Test universal market standards comparison for ANY contract type
    """
    print(f"🎯 Initializing Universal Market Comparator for {contract_type.value}...")
    
    try:
        # Initialize universal market comparator
        model_loader      = ModelLoader()
        market_comparator = UniversalMarketComparator(model_loader  = model_loader,
                                                      contract_type = contract_type,
                                                     )
        
        # Run universal market comparison
        print("🔍 Comparing clauses to universal market standards...")
        comparisons = market_comparator.compare_to_market(clauses         = clauses,
                                                          max_comparisons = 50,
                                                         )
        
        # Display results
        print(f"\n📈 UNIVERSAL MARKET COMPARISON RESULTS:")
        print(f"   Contract Type: {contract_type.value}")
        print(f"   Total Comparisons: {len(comparisons)}")
        
        if comparisons:
            # Assessment summary
            summary = market_comparator.get_assessment_summary(comparisons)
            
            print(f"\n📊 ASSESSMENT SUMMARY:")
            print(f"   Aggressive Terms: {summary['assessments']['aggressive']}")
            print(f"   Unfavorable Terms: {summary['assessments']['unfavorable']}")
            print(f"   Standard Terms: {summary['assessments']['standard']}")
            print(f"   Favorable Terms: {summary['assessments']['favorable']}")
            print(f"   Average Similarity: {summary['average_similarity']:.3f}")
            print(f"   Categories Analyzed: {', '.join(summary['categories_analyzed'][:5])}")
            
            # Show high-risk comparisons
            high_risk = market_comparator.get_high_risk_comparisons(comparisons)
            
            print(f"\n🚨 HIGH-RISK MARKET COMPARISONS:")
            for i, comparison in enumerate(high_risk[:5]):
                print(f"\n   {i+1}. [{comparison.clause_category}] - {comparison.assessment.upper()}")
                print(f"      Original Category: {comparison.original_category}")
                print(f"      Similarity: {comparison.similarity_score:.3f}")
                print(f"      Explanation: {comparison.explanation}")
                
                if comparison.recommendation:
                    print(f"      💡 Recommendation: {comparison.recommendation}")
            
            # Show sample comparisons
            print(f"\n🔍 SAMPLE COMPARISONS:")
            for i, comparison in enumerate(comparisons[:3]):
                print(f"\n   {i+1}. [{comparison.clause_category}] - {comparison.assessment}")
                print(f"      Original: {comparison.original_category}")
                print(f"      Your Clause: {comparison.user_clause[:80]}...")
                print(f"      Market Standard: {comparison.market_standard[:80]}...")
                print(f"      Similarity Score: {comparison.similarity_score:.3f}")
        else:
            print("❌ No comparisons found. This might indicate:")
            print("   - Clause categories don't match any market standards")
            print("   - Embedding model issues")
            print("   - Very unique/unusual contract terms")
        
        return comparisons
        
    except Exception as e:
        print(f"❌ Universal market comparison failed: {e}")
        import traceback
        traceback.print_exc()
        return None

# Run universal market comparator test
universal_market_results = test_universal_market_comparator(clauses       = extracted_clauses,
                                                            contract_type = ContractType.EMPLOYMENT, # This can be ANY contract type!
                                                           )



🌍 STEP 12: Testing Universal Market Comparator
------------------------------------------------------------
🎯 Initializing Universal Market Comparator for employment...
🔍 Comparing clauses to universal market standards...

📈 UNIVERSAL MARKET COMPARISON RESULTS:
   Contract Type: employment
   Total Comparisons: 0
❌ No comparisons found. This might indicate:
   - Clause categories don't match any market standards
   - Embedding model issues
   - Very unique/unusual contract terms


## Complete Service Integration Test

In [ ]:
print("\n🚀 STEP 13: Complete Analysis Pipeline")
print("=" * 60)

def complete_contract_analysis(file_path, use_ai = True):
    """
    Complete end-to-end contract analysis
    """
    print("🎯 Starting Complete Contract Analysis Pipeline...")
    
    # Initialize logging
    ContractAnalyzerLogger.setup(log_dir  = CONFIG["log_directory"], 
                                 app_name = "complete_analysis",
                                )
    
    analysis_results = {'file_info'           : {},
                        'validation'          : {},
                        'processing'          : {},
                        'classification'      : {},
                        'clause_extraction'   : {},
                        'risk_analysis'       : {},
                        'term_analysis'       : {},
                        'protection_analysis' : {},
                        'market_comparison'   : {},
                        'llm_interpretation'  : {},
                        'negotiation_strategy': {},
                        'llm_analysis'        : {},
                       }
    
    try:
        # Step 1: Document Reading
        print("\n📄 1. Document Reading...")
        reader                        = DocumentReader()
        file_contents                 = reader.read_file(file_path, "pdf")
        text                          = file_contents.get('text', '') if isinstance(file_contents, dict) else str(file_contents)
        
        analysis_results['file_info'] = {'text_length'        : len(text),
                                         'file_type'          : 'pdf',
                                         'extraction_success' : bool(text.strip()),
                                        }
        
        # Step 2: Contract Validation
        print("🔍 2. Contract Validation...")
        validator                          = ContractValidator()
        is_contract, val_type, val_message = validator.is_valid_contract(text)
        val_report                         = validator.get_validation_report(text)
        
        analysis_results['validation']     = {'is_contract'        : is_contract,
                                              'confidence_level'   : val_type,
                                              'validation_message' : val_message,
                                              'score'              : val_report['scores']['total'],
                                              'key_indicators'     : val_report['found_indicators'],
                                             }
        
        # Step 3: Text Processing
        print("📝 3. Text Processing...")
        processor                      = TextProcessor(use_spacy = CONFIG["use_spacy"])
        stats                          = processor.get_text_statistics(text)
        entities                       = processor.extract_legal_entities(text)
        chunks                         = processor.chunk_text_for_embedding(text)
        
        analysis_results['processing'] = {'statistics'      : stats,
                                          'entity_counts'   : {k: len(v) for k, v in entities.items()},
                                          'key_entities'    : {'parties' : entities.get('parties', []),
                                                               'dates'   : entities.get('dates', []),
                                                               'amounts' : entities.get('amounts', [])
                                                              },
                                          'analysis_chunks' : len(chunks),
                                         }
        
        # Step 4: AI-Powered Analysis (Optional)
        if use_ai:
            print("🤖 4. AI-Powered Analysis...")
            try:
                # Contract Classification
                model_loader                       = ModelLoader()
                classifier                         = ContractClassifier(model_loader)
                classification                     = classifier.classify_contract(text)
                
                analysis_results['classification'] = {'primary_category' : classification.category,
                                                      'subcategory'      : classification.subcategory,
                                                      'confidence'       : classification.confidence,
                                                      'reasoning'        : classification.reasoning,
                                                     }
                
                # Clause Extraction
                print("🔍 5. Clause Extraction...")
                clause_extractor                      = ClauseExtractor(model_loader      = model_loader,
                                                                        contract_category = classification.category, 
                                                                       )
                clauses                               = clause_extractor.extract_clauses(contract_text = text, 
                                                                                         max_clauses   = 50,
                                                                                        )
                clause_stats                          = clause_extractor.get_extraction_stats(clauses)
                risky_clauses                         = clause_extractor.get_high_risk_clauses(clauses)
                
                analysis_results['clause_extraction'] = {'total_clauses'       : len(clauses),
                                                         'categories_found'    : list(set(c.category for c in clauses)),
                                                         'risky_clauses_count' : len(risky_clauses),
                                                         'avg_confidence'      : clause_stats['avg_confidence'],
                                                         'extraction_stats'    : clause_stats,
                                                         'sample_clauses'      : [{'reference'       : c.reference,
                                                                                   'category'        : c.category,
                                                                                   'confidence'      : c.confidence,
                                                                                   'risk_indicators' : c.risk_indicators,
                                                                                  } for c in clauses
                                                                                 ],
                                                        }
                
                # Risk Analysis
                print("📊 6. Risk Analysis...")
                risk_analyzer                         = MultiFactorRiskAnalyzer(contract_type = ContractType.EMPLOYMENT)
                risk_score                            = risk_analyzer.analyze_risk(contract_text = text,
                                                                                   clauses       = clauses,
                                                                                  )
                
                analysis_results['risk_analysis']     = {'overall_score'     : risk_score.overall_score,
                                                         'risk_level'        : risk_score.risk_level,
                                                         'category_scores'   : risk_score.category_scores,
                                                         'risk_factors'      : risk_score.risk_factors,
                                                         'benchmark_results' : risk_score.benchmark_comparison,
                                                        }
                
                # Term Analysis
                print("⚖️  7. Term Analysis...")
                term_analyzer                         = TermAnalyzer()
                unfavorable_terms                     = term_analyzer.analyze_unfavorable_terms(contract_text = text,
                                                                                                clauses       = clauses,
                                                                                               )
                severity_dist                         = term_analyzer.get_severity_distribution(unfavorable_terms)
                category_dist                         = term_analyzer.get_category_distribution(unfavorable_terms)
                
                analysis_results['term_analysis']     = {'total_terms'       : len(unfavorable_terms),
                                                         'severity_dist'     : severity_dist,
                                                         'category_dist'     : category_dist,
                                                         'critical_terms'    : [t for t in unfavorable_terms if t.severity == "critical"],
                                                         'sample_terms'      : unfavorable_terms[:5],
                                                        }
                
                # Protection Analysis
                print("🛡️  8. Protection Analysis...")
                protection_checker                    = ProtectionChecker()
                missing_protections                   = protection_checker.check_missing_protections(contract_text = text,
                                                                                                     clauses       = clauses,
                                                                                                    )
                importance_dist                       = protection_checker.get_importance_distribution(missing_protections)
                critical_protections                  = protection_checker.get_critical_missing(missing_protections)
                
                analysis_results['protection_analysis'] = {'total_missing'      : len(missing_protections),
                                                           'importance_dist'    : importance_dist,
                                                           'critical_protections': critical_protections,
                                                           'sample_protections' : missing_protections[:5],
                                                          }
                
                # Market Comparison
                print("📈 9. Market Comparison...")
                market_comparator                     = UniversalMarketComparator(model_loader = model_loader)
                market_comparisons                    = market_comparator.compare_to_market(clauses        = clauses,
                                                                                            max_comparisons = 15,
                                                                                           )
                market_summary                        = market_comparator.get_assessment_summary(market_comparisons)
                high_risk_market                      = market_comparator.get_high_risk_comparisons(market_comparisons)
                
                analysis_results['market_comparison'] = {'total_comparisons'  : len(market_comparisons),
                                                         'assessment_summary' : market_summary,
                                                         'high_risk_count'    : len(high_risk_market),
                                                         'sample_comparisons' : market_comparisons[:3],
                                                        }
                
                # LLM Interpretation (if available)
                llm_manager = LLMManager(default_provider = LLMProvider.OLLAMA)
                
                if LLMProvider.OLLAMA in llm_manager.get_available_providers():
                    print("💬 10. LLM Interpretation...")
                    try:
                        llm_interpreter                  = LLMClauseInterpreter(llm_manager = llm_manager)
                        interpretations                  = llm_interpreter.interpret_clauses(clauses     = clauses[:5],
                                                                                             max_clauses = 5,
                                                                                            )
                        unfavorable_interpretations      = llm_interpreter.get_unfavorable_interpretations(interpretations)
                        high_risk_interpretations        = llm_interpreter.get_high_risk_interpretations(interpretations)
                        
                        analysis_results['llm_interpretation'] = {'total_interpreted'        : len(interpretations),
                                                                  'unfavorable_count'       : len(unfavorable_interpretations),
                                                                  'high_risk_count'         : len(high_risk_interpretations),
                                                                  'sample_interpretations'  : interpretations[:3],
                                                                 }
                    except Exception as e:
                        print(f"⚠️  LLM interpretation failed: {e}")
                        analysis_results['llm_interpretation'] = {'error': str(e)}
                
                # Negotiation Strategy
                print("🤝 11. Negotiation Strategy...")
                try:
                    negotiation_engine                  = NegotiationEngine(llm_manager = llm_manager)
                    negotiation_points                  = negotiation_engine.generate_negotiation_points(risk_analysis       = risk_score,
                                                                                                         unfavorable_terms   = unfavorable_terms,
                                                                                                         missing_protections = missing_protections,
                                                                                                         clauses             = clauses,
                                                                                                         max_points          = 10,
                                                                                                        )
                    critical_points                     = negotiation_engine.get_critical_points(negotiation_points)
                    strategy_doc                        = negotiation_engine.generate_negotiation_strategy_document(negotiation_points)
                    
                    analysis_results['negotiation_strategy'] = {'total_points'     : len(negotiation_points),
                                                                'critical_points'  : len(critical_points),
                                                                'strategy_doc_len' : len(strategy_doc),
                                                                'sample_points'    : negotiation_points[:3],
                                                               }
                except Exception as e:
                    print(f"⚠️  Negotiation strategy failed: {e}")
                    analysis_results['negotiation_strategy'] = {'error': str(e)}
                
                # LLM Summary Analysis
                print("💬 12. LLM Summary Analysis...")
                if LLMProvider.OLLAMA in llm_manager.get_available_providers():
                    try:
                        # Create comprehensive summary using all analyses
                        summary_context = f"""
Contract Type: {classification.category}
Risk Score: {risk_score.overall_score}/100 ({risk_score.risk_level})
Unfavorable Terms: {len(unfavorable_terms)} (Critical: {severity_dist.get('critical', 0)})
Missing Protections: {len(missing_protections)} (Critical: {importance_dist.get('critical', 0)})
Market Comparison: {market_summary['assessments']['aggressive']} aggressive terms, {market_summary['assessments']['unfavorable']} unfavorable terms

Key Risk Factors: {', '.join(risk_score.risk_factors[:3])}
Top Unfavorable Terms: {', '.join([t.term for t in unfavorable_terms[:3]])}
Critical Missing Protections: {', '.join([p.protection for p in critical_protections[:2]])}
Aggressive Market Terms: {', '.join([c.clause_category for c in high_risk_market[:2]])}
"""
                        
                        summary_prompt = f"""
Based on this comprehensive contract analysis, provide a concise executive summary:

{summary_context}

Provide a 3-4 bullet point summary highlighting:
1. Overall risk level and key concerns
2. Most critical unfavorable terms to negotiate
3. Essential missing protections to add
4. Market deviations that need attention
5. Recommended negotiation priorities

Keep it business-friendly and actionable.
"""
                        
                        summary_response = llm_manager.complete(prompt      = summary_prompt,
                                                                max_tokens  = 1024,
                                                                temperature = 0.1,
                                                               )
                        
                        analysis_results['llm_analysis'] = {'summary'                : summary_response.text if summary_response.success else "LLM analysis failed",
                                                            'provider'               : 'ollama',
                                                            'context_used'           : summary_context,
                                                           }
                    except Exception as e:
                        print(f"⚠️  LLM summary analysis failed: {e}")
                        analysis_results['llm_analysis'] = {'error': str(e)}
                
            except Exception as e:
                print(f"⚠️  AI analysis partially failed: {e}")
                analysis_results['classification']      = {'error': str(e)}
                analysis_results['clause_extraction']   = {'error': str(e)}
                analysis_results['risk_analysis']       = {'error': str(e)}
                analysis_results['term_analysis']       = {'error': str(e)}
                analysis_results['protection_analysis'] = {'error': str(e)}
                analysis_results['market_comparison']   = {'error': str(e)}
                analysis_results['llm_interpretation']  = {'error': str(e)}
                analysis_results['negotiation_strategy']= {'error': str(e)}
                analysis_results['llm_analysis']        = {'error': str(e)}
        
        print("✅ Analysis completed successfully!")
        return analysis_results
        
    except Exception as e:
        print(f"❌ Analysis failed: {e}")
        return {'error': str(e)}

# Run complete analysis
print("🔧 Running complete analysis (this may take a few minutes)...")
complete_results = complete_contract_analysis(file_path = CONFIG["pdf_file_path"], 
                                              use_ai    = True,
                                             )


# Display results
print("\n" + "=" * 60)
print("📊 COMPLETE ANALYSIS RESULTS")
print("=" * 60)

if ('error' in complete_results):
    print(f"❌ Error: {complete_results['error']}")

else:
    # File Info
    file_info = complete_results['file_info']
    print(f"📄 FILE INFO:")
    print(f"   Text Length: {file_info['text_length']:,} characters")
    print(f"   Extraction Success: {file_info['extraction_success']}")
    
    # Validation
    validation = complete_results['validation']
    print(f"\n🔍 VALIDATION:")
    print(f"   Is Contract: {validation['is_contract']}")
    print(f"   Confidence: {validation['confidence_level']}")
    print(f"   Score: {validation['score']}")
    print(f"   Key Indicators: {', '.join(validation['key_indicators'][:3])}")
    
    # Processing
    processing = complete_results['processing']
    print(f"\n📝 PROCESSING:")
    print(f"   Sentences: {processing['statistics']['sentence_count']}")
    print(f"   Words: {processing['statistics']['word_count']}")
    print(f"   Language: {processing['statistics']['language']}")
    print(f"   Parties Found: {processing['entity_counts'].get('parties', 0)}")
    print(f"   Dates Found: {processing['entity_counts'].get('dates', 0)}")
    
    # Classification (if available)
    if ((complete_results['classification']) and ('primary_category' in complete_results['classification'])):
        classification = complete_results['classification']
        print(f"\n🏷️  CLASSIFICATION:")
        print(f"   Category: {classification['primary_category']}")
        print(f"   Subcategory: {classification['subcategory']}")
        print(f"   Confidence: {classification['confidence']:.2f}")
        print(f"   Key Reasoning: {classification['reasoning'][0] if classification['reasoning'] else 'N/A'}")
    
    # Clause Extraction (if available)
    if ((complete_results['clause_extraction']) and ('total_clauses' in complete_results['clause_extraction'])):
        clause_extraction = complete_results['clause_extraction']
        print(f"\n🔍 CLAUSE EXTRACTION:")
        print(f"   Total Clauses: {clause_extraction['total_clauses']}")
        print(f"   Categories Found: {', '.join(clause_extraction['categories_found'][:5])}")
        print(f"   Risky Clauses: {clause_extraction['risky_clauses_count']}")
        print(f"   Average Confidence: {clause_extraction['avg_confidence']:.3f}")
        
        # Show sample clauses
        if (clause_extraction['sample_clauses']):
            print(f"   Sample Clauses:")
            for i, clause in enumerate(clause_extraction['sample_clauses'][:2]):
                print(f"     {i+1}. [{clause['category']}] {clause['reference']}")
                print(f"        Confidence: {clause['confidence']:.3f}")
                if clause['risk_indicators']:
                    print(f"        ⚠️  Risks: {clause['risk_indicators']}")
    
    # Risk Analysis (if available)
    if ((complete_results['risk_analysis']) and ('overall_score' in complete_results['risk_analysis'])):
        risk_analysis = complete_results['risk_analysis']
        print(f"\n📊 RISK ANALYSIS:")
        print(f"   Overall Score: {risk_analysis['overall_score']}/100")
        print(f"   Risk Level: {risk_analysis['risk_level']}")
        print(f"   High-Risk Categories: {len(risk_analysis['risk_factors'])}")
        print(f"   Key Risk Factors: {', '.join(risk_analysis['risk_factors'][:3])}")
    
    # Term Analysis (if available)
    if ((complete_results['term_analysis']) and ('total_terms' in complete_results['term_analysis'])):
        term_analysis = complete_results['term_analysis']
        print(f"\n⚖️  TERM ANALYSIS:")
        print(f"   Unfavorable Terms: {term_analysis['total_terms']}")
        print(f"   Critical Terms: {term_analysis['severity_dist'].get('critical', 0)}")
        print(f"   High Terms: {term_analysis['severity_dist'].get('high', 0)}")
        
        if term_analysis['sample_terms']:
            print(f"   Sample Critical Terms:")
            for i, term in enumerate(term_analysis['sample_terms'][:2]):
                print(f"     {i+1}. {term.term} ({term.severity})")
    
    # Protection Analysis (if available)
    if ((complete_results['protection_analysis']) and ('total_missing' in complete_results['protection_analysis'])):
        protection_analysis = complete_results['protection_analysis']
        print(f"\n🛡️  PROTECTION ANALYSIS:")
        print(f"   Missing Protections: {protection_analysis['total_missing']}")
        print(f"   Critical Missing: {protection_analysis['importance_dist'].get('critical', 0)}")
        print(f"   High Missing: {protection_analysis['importance_dist'].get('high', 0)}")
        
        if protection_analysis['critical_protections']:
            print(f"   Critical Missing Protections:")
            for i, protection in enumerate(protection_analysis['critical_protections'][:2]):
                print(f"     {i+1}. {protection.protection}")
    
    # Market Comparison (if available)
    if ((complete_results['market_comparison']) and ('total_comparisons' in complete_results['market_comparison'])):
        market_comparison = complete_results['market_comparison']
        print(f"\n📈 MARKET COMPARISON:")
        print(f"   Total Comparisons: {market_comparison['total_comparisons']}")
        if 'assessment_summary' in market_comparison:
            summary = market_comparison['assessment_summary']
            print(f"   Aggressive Terms: {summary['assessments']['aggressive']}")
            print(f"   Unfavorable Terms: {summary['assessments']['unfavorable']}")
            print(f"   Standard Terms: {summary['assessments']['standard']}")
            print(f"   Favorable Terms: {summary['assessments']['favorable']}")
            print(f"   Average Similarity: {summary['average_similarity']:.3f}")
        
        if market_comparison['sample_comparisons']:
            print(f"   Sample Market Comparisons:")
            for i, comparison in enumerate(market_comparison['sample_comparisons'][:2]):
                print(f"     {i+1}. [{comparison.clause_category}] - {comparison.assessment}")
                print(f"        Similarity: {comparison.similarity_score:.3f}")
    
    # Negotiation Strategy (if available)
    if ((complete_results['negotiation_strategy']) and ('total_points' in complete_results['negotiation_strategy'])):
        negotiation_strategy = complete_results['negotiation_strategy']
        print(f"\n🤝 NEGOTIATION STRATEGY:")
        print(f"   Total Points: {negotiation_strategy['total_points']}")
        print(f"   Critical Points: {negotiation_strategy['critical_points']}")
        
        if negotiation_strategy['sample_points']:
            print(f"   Sample Negotiation Points:")
            for i, point in enumerate(negotiation_strategy['sample_points'][:2]):
                print(f"     {i+1}. {point.issue} (Priority: {point.priority})")
    
    # LLM Analysis (if available)
    if ((complete_results['llm_analysis']) and ('summary' in complete_results['llm_analysis'])):
        llm_analysis = complete_results['llm_analysis']
        print(f"\n💬 EXECUTIVE SUMMARY:")
        print(f"   {llm_analysis['summary']}")
        